# Notebook for visualizations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import os

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, layout
from bokeh.models import BasicTicker, BooleanFilter, CDSView, ColorBar, ColumnDataSource, FactorRange, GroupFilter, Legend, LinearColorMapper, LabelSet, PrintfTickFormatter, Span, LinearAxis, Range1d 
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral6, Spectral11, Spectral5, Viridis256, Colorblind, Magma256, Turbo256
from bokeh.transform import transform
from IPython.display import display, Math
from scipy import optimize, stats
from math import exp, cos, sqrt, pi, sin, sqrt
from sklearn.metrics import mean_squared_error
from typing import List

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

mypalette = sns.color_palette(['#51ffda']) # https://www.hsluv.org/
sns.set_palette(mypalette)

output_notebook()

In [ ]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()


## Visualization of curve fitting

In [ ]:
ospitaletto = AVAILABLE_DATASETS[OspitalettoDataset.OSPITALETTO]

miami = AVAILABLE_DATASETS[NOAA2010Dataset.MIAMI_FL].reset_index(drop=False)
fresno = AVAILABLE_DATASETS[NOAA2010Dataset.FRESNO_CA].reset_index(drop=False)
olympia = AVAILABLE_DATASETS[NOAA2010Dataset.OLYMPIA_WA].reset_index(drop=False)
rochester = AVAILABLE_DATASETS[NOAA2010Dataset.ROCHESTER_NY].reset_index(drop=False)

london = AVAILABLE_DATASETS[InsPireDataset.LONDON_UK].reset_index(drop=False)
madrid = AVAILABLE_DATASETS[InsPireDataset.MADRID_SPA].reset_index(drop=False)
rome = AVAILABLE_DATASETS[InsPireDataset.ROME_IT].reset_index(drop=False)
stuttgart = AVAILABLE_DATASETS[InsPireDataset.STUTTGART_GER].reset_index(drop=False)

In [ ]:
ospitaletto["city_key"] = "ospitaletto"
ospitaletto_source = ColumnDataSource(ospitaletto)

us_data = miami.append(other=fresno).append(other=olympia).append(other=rochester)
us_source = ColumnDataSource(us_data)

europe_data = london.append(other=madrid).append(other=rome).append(other=stuttgart)
europe_source = ColumnDataSource(europe_data)

In [ ]:
miami_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.MIAMI_FL)])
fresno_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.FRESNO_CA)])
olympia_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.OLYMPIA_WA)])
rochester_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.ROCHESTER_NY)])

miami_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "winter"))])
fresno_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "winter"))])
olympia_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "winter"))])
rochester_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "winter"))])

miami_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "spring"))])
fresno_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "spring"))])
olympia_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "spring"))])
rochester_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "spring"))])

miami_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "summer"))])
fresno_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "summer"))])
olympia_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "summer"))])
rochester_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "summer"))])

miami_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "fall"))])
fresno_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "fall"))])
olympia_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "fall"))])
rochester_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "fall"))])

london_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.LONDON_UK)])
madrid_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.MADRID_SPA)])
rome_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.ROME_IT)])
stuttgart_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.STUTTGART_GER)])

london_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "winter"))])
madrid_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "winter"))])
rome_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "winter"))])
stuttgart_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "winter"))])

london_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "spring"))])
madrid_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "spring"))])
rome_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "spring"))])
stuttgart_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "spring"))])

london_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "summer"))])
madrid_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "summer"))])
rome_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "summer"))])
stuttgart_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "summer"))])

london_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "fall"))])
madrid_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "fall"))])
rome_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "fall"))])
stuttgart_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "fall"))])



In [ ]:
color_map = {
    "miami": "#2c53d2", #navy
    "fresno": "#b30000", #red
    "olympia": "#00b300", #green
    "rochester": "#8296a5", #gray
    
    "london": "#2c53d2",  #navy
    "madrid": "#b30000", #red
    "rome": "#00b300", #green
    "stuttgart": "#8296a5", #gray
    
    # "winter": '#1212be', #blue
    # "spring": '#c83258', #pink
    # "summer": '#c81313', #red
    # "fall":  '#4d4dff', #blue
    
    "winter": '#2c53d2', #blue
    "spring": '#00b300', #green
    "summer": '#b30000', #red
    "fall":  '#ef9331', #orange
}

#More colors in https://colors.muz.li/

In [ ]:
# The width on these plots is equivalent to ms in datetime, i.e., width = 1 === width = 1ms
ONE_DAY_WIDTH = 1 * 24 * 60 * 60 * 1000

## USA Data

We acquired the data from the National Oceanic and Atmospheric Administration (NOAA). The data included information of different cities across the United States with respect to their latitude, longitude, stations, air temperature, humidity, and many more. We solely used the air temperature and selected four different cities, namely: 

- Miami: located in the state of Florida, South-East of the country.
- Fresno: located in the state of Florida, South-West of the country.
- Olympia: located in the state of Washington, North-West of the country.
- Rochester: located in the state of New York, North-East of the country.

Located on the map, they look like this:

<img src="./images/USA cities.png">    

The U.S. Hourly Climate Normals for 1981 to 2010 are 30-year averages of meteorological parameters for thousands of U.S. stations located across the 50 states, as well as U.S. territories, commonwealths, the Compact of Free Association nations, and one station inCanada. NOAA Climate Normals are a large suite of data products that provide users with many tools to understand typical climate conditions for thousands of locations across the United States. As many NWS stations as possible are used, including those from the NWS Cooperative Observer Program (COOP) Network as well as some additional stations that have a Weather Bureau Army-Navy (WBAN) station identification number, including stations from the Climate Reference Network (CRN). The comprehensive U.S. Climate Normals dataset includes various derived products including daily air temperature normals (including maximum and minimum temperature normal, heating and cooling degree day normal, and others), precipitation normals (including snowfall and snow depth, percentiles, frequencies and other), and hourly normals (all normal derived from hourly data including temperature, dew point, heat index, wind chill, wind, cloudiness, heating and cooling degree hours, pressure normals). Users can access the data either by product or by station. Included in the dataset is extensive documentation to describe station metadata, filename descriptions, and methodology of producing the data. All data utilized in the computation of the 1981-2010 Climate Normals were taken from the ISD Lite (a subset of derived Integrated Surface Data), the Global Historical Climatology Network-Daily dataset, and standardized monthly temperature data (COOP). These source datasets (including intermediate datasets used in the computation of products) are also archived at the NOAA NCDC. [More here on the NOAA website](https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00824/html)




## Europe Data

We acquired the data from the InsPire project. We solely used the air temperature and selected four different cities, namely: 

- Rome: located in Italy.
- Madrid: located in Spain.
- Stuttgart: located in Germany.
- London: located in the United Kingdom.

Located on the map, they look like this:

<img src="./images/Europe_cities.png">      


## Air Temperature vs Fitted Air Temperature in USA

In [ ]:
max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",
                                y_range=(min_temp, max_temp),)
miami_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view)
miami_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view, size=1)


fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                 title="Fresno, California, USA",
                                 y_range=miami_air_temp_and_fit.y_range,
                                 x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
fresno_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["fresno"], alpha=0.4, source=us_source, view=fresno_view, size=1)

olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                  title="Olympia, Washington, USA",
                                  y_range=miami_air_temp_and_fit.y_range,
                                  x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
olympia_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["olympia"], alpha=0.4, source=us_source, view=olympia_view, size=1)

rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                    title="Rochester, New York, USA",
                                    y_range=miami_air_temp_and_fit.y_range,
                                    x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
rochester_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["rochester"], alpha=0.4, source=us_source, view=rochester_view, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                           title="Adjusted air temperature",
                           y_range=miami_air_temp_and_fit.y_range,
                           x_range=miami_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Miami", color=color_map["miami"], alpha=0.7, source=us_source, view=miami_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Fresno", color=color_map["fresno"], alpha=0.7, source=us_source, view=fresno_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Olympia", color=color_map["olympia"], alpha=0.7, source=us_source, view=olympia_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Rochester", color=color_map["rochester"], alpha=0.7, source=us_source, view=rochester_view, line_width=3)

mapper = LinearColorMapper(palette=Spectral11, low=us_data["air_temp"].min(), high=us_data["air_temp"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

miami_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="Miami, FL - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
miami_air_temp_heatmap.grid.visible = False
miami_air_temp_heatmap.axis.axis_line_color = None
miami_air_temp_heatmap.axis.major_tick_line_color = None
miami_air_temp_heatmap.axis.major_label_text_font_size = "7px"
miami_air_temp_heatmap.axis.major_label_standoff = 0
miami_air_temp_heatmap.xaxis.major_label_orientation = 1.0

fresno_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="Fresno, CA - Air Temperature ", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
fresno_air_temp_heatmap.grid.visible = False
fresno_air_temp_heatmap.axis.axis_line_color = None
fresno_air_temp_heatmap.axis.major_tick_line_color = None
fresno_air_temp_heatmap.axis.major_label_text_font_size = "7px"
fresno_air_temp_heatmap.axis.major_label_standoff = 0
fresno_air_temp_heatmap.xaxis.major_label_orientation = 1.0

olympia_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="Olympia, WA - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
olympia_air_temp_heatmap.grid.visible = False
olympia_air_temp_heatmap.axis.axis_line_color = None
olympia_air_temp_heatmap.axis.major_tick_line_color = None
olympia_air_temp_heatmap.axis.major_label_text_font_size = "7px"
olympia_air_temp_heatmap.axis.major_label_standoff = 0
olympia_air_temp_heatmap.xaxis.major_label_orientation = 1.0

rochester_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="Rochester, NY - Air Temperature ", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
rochester_air_temp_heatmap.grid.visible = False
rochester_air_temp_heatmap.axis.axis_line_color = None
rochester_air_temp_heatmap.axis.major_tick_line_color = None
rochester_air_temp_heatmap.axis.major_label_text_font_size = "7px"
rochester_air_temp_heatmap.axis.major_label_standoff = 0
rochester_air_temp_heatmap.xaxis.major_label_orientation = 1.0

miami_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=miami_view, line_color=None, fill_color=transform('air_temp', mapper))
miami_air_temp_heatmap.add_layout(color_bar, 'right')

fresno_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=fresno_view, line_color=None, fill_color=transform('air_temp', mapper))
fresno_air_temp_heatmap.add_layout(color_bar, 'right')

olympia_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=olympia_view, line_color=None, fill_color=transform('air_temp', mapper))
olympia_air_temp_heatmap.add_layout(color_bar, 'right')

rochester_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=rochester_view, line_color=None, fill_color=transform('air_temp', mapper))
rochester_air_temp_heatmap.add_layout(color_bar, 'right')

show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
    [only_air_temp_fit],
    [miami_air_temp_heatmap, fresno_air_temp_heatmap,],
    [olympia_air_temp_heatmap, rochester_air_temp_heatmap],
]))

## Air Temperature vs Fitted Air Temperature in Europe

In [ ]:
max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="London, United Kingdom",
                                y_range=(min_temp, max_temp),)
london_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
london_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view, size=1)

madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
madrid_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view, size=1)

rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
rome_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view, size=1)

stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
stuttgart_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                                title="Adjusted air temperature",
                          y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="London, UK", color=color_map["london"], alpha=0.7, source=europe_source, view=london_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.7, source=europe_source, view=stuttgart_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.7, source=europe_source, view=rome_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.7, source=europe_source, view=madrid_view, line_width=3)

mapper = LinearColorMapper(palette=Spectral11, low=europe_data["air_temp"].min(), high=europe_data["air_temp"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

london_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="London, UK - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
london_air_temp_heatmap.grid.visible = False
london_air_temp_heatmap.axis.axis_line_color = None
london_air_temp_heatmap.axis.major_tick_line_color = None
london_air_temp_heatmap.axis.major_label_text_font_size = "7px"
london_air_temp_heatmap.axis.major_label_standoff = 0
london_air_temp_heatmap.xaxis.major_label_orientation = 1.0

sttutgart_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="Stuttgart, Germany - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
sttutgart_air_temp_heatmap.grid.visible = False
sttutgart_air_temp_heatmap.axis.axis_line_color = None
sttutgart_air_temp_heatmap.axis.major_tick_line_color = None
sttutgart_air_temp_heatmap.axis.major_label_text_font_size = "7px"
sttutgart_air_temp_heatmap.axis.major_label_standoff = 0
sttutgart_air_temp_heatmap.xaxis.major_label_orientation = 1.0

rome_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="Rome, Italy - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
rome_air_temp_heatmap.grid.visible = False
rome_air_temp_heatmap.axis.axis_line_color = None
rome_air_temp_heatmap.axis.major_tick_line_color = None
rome_air_temp_heatmap.axis.major_label_text_font_size = "7px"
rome_air_temp_heatmap.axis.major_label_standoff = 0
rome_air_temp_heatmap.xaxis.major_label_orientation = 1.0

madrid_air_temp_heatmap = figure(plot_width=600, plot_height=300, title="Madrid, Spain - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
madrid_air_temp_heatmap.grid.visible = False
madrid_air_temp_heatmap.axis.axis_line_color = None
madrid_air_temp_heatmap.axis.major_tick_line_color = None
madrid_air_temp_heatmap.axis.major_label_text_font_size = "7px"
madrid_air_temp_heatmap.axis.major_label_standoff = 0
madrid_air_temp_heatmap.xaxis.major_label_orientation = 1.0

london_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=london_view, line_color=None, fill_color=transform('air_temp', mapper))
london_air_temp_heatmap.add_layout(color_bar, 'right')

sttutgart_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=stuttgart_view, line_color=None, fill_color=transform('air_temp', mapper))
sttutgart_air_temp_heatmap.add_layout(color_bar, 'right')

rome_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=rome_view, line_color=None, fill_color=transform('air_temp', mapper))
rome_air_temp_heatmap.add_layout(color_bar, 'right')

madrid_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=madrid_view, line_color=None, fill_color=transform('air_temp', mapper))
madrid_air_temp_heatmap.add_layout(color_bar, 'right')


show(layout([
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],
    [only_air_temp_fit],
    [london_air_temp_heatmap, sttutgart_air_temp_heatmap],
    [rome_air_temp_heatmap, madrid_air_temp_heatmap],
]))

## Visualization of Air Temp and Ground Temp in USA


In [ ]:
max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",
                                y_range=(min_temp, max_temp),)
miami_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)
miami_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["miami"], alpha=0.1, source=us_source, view=miami_view, size=1)


fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Fresno, California, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
fresno_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["fresno"], alpha=0.1, source=us_source, view=fresno_view, size=1)


olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Olympia, Washington, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
olympia_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["olympia"], alpha=1, source=us_source, view=olympia_view, size=1)


rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Rochester, New York, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
rochester_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["rochester"], alpha=0.1, source=us_source, view=rochester_view, size=1)


ground_temp = figure(plot_width=1200, 
                     plot_height=300, 
                     x_axis_type="datetime",
                     title="Ground temperature - 1m depth",
                     y_range=london_air_temp_and_fit.y_range,
                     x_range=london_air_temp_and_fit.x_range)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Miami, FL", color=color_map["miami"], alpha=0.7, source=us_source, view=miami_view, line_width=3)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Fresno, CA", color=color_map["fresno"], alpha=0.7, source=us_source, view=fresno_view, line_width=3)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Olympia, WA", color=color_map["olympia"], alpha=0.7, source=us_source, view=olympia_view, line_width=3)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Rochester, NY", color=color_map["rochester"], alpha=0.7, source=us_source, view=rochester_view, line_width=3)

mapper = LinearColorMapper(palette=Spectral11, low=us_data["ground_temp"].min(), high=us_data["ground_temp"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

miami_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="Miami, FL - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
miami_ground_temp_heatmap.grid.visible = False
miami_ground_temp_heatmap.axis.axis_line_color = None
miami_ground_temp_heatmap.axis.major_tick_line_color = None
miami_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
miami_ground_temp_heatmap.axis.major_label_standoff = 0
miami_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

fresno_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="Fresno, CA - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
fresno_ground_temp_heatmap.grid.visible = False
fresno_ground_temp_heatmap.axis.axis_line_color = None
fresno_ground_temp_heatmap.axis.major_tick_line_color = None
fresno_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
fresno_ground_temp_heatmap.axis.major_label_standoff = 0
fresno_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

olympia_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="Olympia, WA - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
olympia_ground_temp_heatmap.grid.visible = False
olympia_ground_temp_heatmap.axis.axis_line_color = None
olympia_ground_temp_heatmap.axis.major_tick_line_color = None
olympia_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
olympia_ground_temp_heatmap.axis.major_label_standoff = 0
olympia_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

rochester_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="Rochester, NY - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
rochester_ground_temp_heatmap.grid.visible = False
rochester_ground_temp_heatmap.axis.axis_line_color = None
rochester_ground_temp_heatmap.axis.major_tick_line_color = None
rochester_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
rochester_ground_temp_heatmap.axis.major_label_standoff = 0
rochester_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

miami_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=miami_view, line_color=None, fill_color=transform('ground_temp', mapper))
miami_ground_temp_heatmap.add_layout(color_bar, 'right')

fresno_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=fresno_view, line_color=None, fill_color=transform('ground_temp', mapper))
fresno_ground_temp_heatmap.add_layout(color_bar, 'right')

olympia_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=olympia_view, line_color=None, fill_color=transform('ground_temp', mapper))
olympia_ground_temp_heatmap.add_layout(color_bar, 'right')

rochester_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=rochester_view, line_color=None, fill_color=transform('ground_temp', mapper))
rochester_ground_temp_heatmap.add_layout(color_bar, 'right')

show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
    [ground_temp],
    [miami_ground_temp_heatmap, fresno_ground_temp_heatmap],
    [olympia_ground_temp_heatmap, rochester_ground_temp_heatmap],
]))

## Visualization of Air Temp, GroundTemp in Europe

In [ ]:
max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="London, United Kingdom",
                                y_range=(min_temp, max_temp),)
london_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
london_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view, size=1)

madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
madrid_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view, size=1)

rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
rome_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view, size=1)

stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
stuttgart_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                                title="Ground temperature - 1m depth",
                          y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="London, UK", color=color_map["london"], alpha=0.7, source=europe_source, view=london_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.7, source=europe_source, view=stuttgart_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.7, source=europe_source, view=rome_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.7, source=europe_source, view=madrid_view, line_width=3)


mapper = LinearColorMapper(palette=Spectral11, low=europe_data["ground_temp"].min(), high=europe_data["ground_temp"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

london_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="London, UK - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
london_ground_temp_heatmap.grid.visible = False
london_ground_temp_heatmap.axis.axis_line_color = None
london_ground_temp_heatmap.axis.major_tick_line_color = None
london_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
london_ground_temp_heatmap.axis.major_label_standoff = 0
london_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

sttutgart_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="Stuttgart, Germany - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
sttutgart_ground_temp_heatmap.grid.visible = False
sttutgart_ground_temp_heatmap.axis.axis_line_color = None
sttutgart_ground_temp_heatmap.axis.major_tick_line_color = None
sttutgart_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
sttutgart_ground_temp_heatmap.axis.major_label_standoff = 0
sttutgart_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

rome_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="Rome, Italy - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
rome_ground_temp_heatmap.grid.visible = False
rome_ground_temp_heatmap.axis.axis_line_color = None
rome_ground_temp_heatmap.axis.major_tick_line_color = None
rome_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
rome_ground_temp_heatmap.axis.major_label_standoff = 0
rome_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

madrid_ground_temp_heatmap = figure(plot_width=600, plot_height=300, title="Madrid, Spain - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", )
madrid_ground_temp_heatmap.grid.visible = False
madrid_ground_temp_heatmap.axis.axis_line_color = None
madrid_ground_temp_heatmap.axis.major_tick_line_color = None
madrid_ground_temp_heatmap.axis.major_label_text_font_size = "7px"
madrid_ground_temp_heatmap.axis.major_label_standoff = 0
madrid_ground_temp_heatmap.xaxis.major_label_orientation = 1.0

london_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=london_view, line_color=None, fill_color=transform('ground_temp', mapper))
london_ground_temp_heatmap.add_layout(color_bar, 'right')

sttutgart_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=stuttgart_view, line_color=None, fill_color=transform('ground_temp', mapper))
sttutgart_ground_temp_heatmap.add_layout(color_bar, 'right')

rome_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=rome_view, line_color=None, fill_color=transform('ground_temp', mapper))
rome_ground_temp_heatmap.add_layout(color_bar, 'right')

madrid_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=madrid_view, line_color=None, fill_color=transform('ground_temp', mapper))
madrid_ground_temp_heatmap.add_layout(color_bar, 'right')


show(layout([
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],
    [only_air_temp_fit],
    [london_ground_temp_heatmap, sttutgart_ground_temp_heatmap],
    [rome_ground_temp_heatmap, madrid_ground_temp_heatmap],
]))

## Visualization of Aquifer and Yearly Mean Temp in USA

In [ ]:
cities = ["Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
statistics = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [miami.aquifer_temp.head(1).item(), 
                          fresno.aquifer_temp.head(1).item(),
                          olympia.aquifer_temp.head(1).item(), 
                          rochester.aquifer_temp.head(1).item()],
         'Mean Air Temp': [miami.air_temp.mean(), 
                           fresno.air_temp.mean(),
                           olympia.air_temp.mean(), 
                           rochester.air_temp.mean()],}

x=[(city, stat) for city in cities for stat in statistics]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["miami"], color_map["miami"], 
                                           color_map["fresno"], color_map["fresno"],
                                           color_map["olympia"], color_map["olympia"], 
                                           color_map["rochester"], color_map["rochester"]],
                                   alpha=[0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,]))

aquifer_temp = figure(plot_width=1200, 
                      plot_height=600, 
                      x_range=FactorRange(*x), 
                      title="Comparison of aquifer and mean air temperatures across cities in the USA")
aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
aquifer_temp.y_range.start = 0
aquifer_temp.x_range.range_padding = 0.1
aquifer_temp.xaxis.major_label_orientation = .9
aquifer_temp.xgrid.grid_line_color = None

show(aquifer_temp)

## Visualization of Aquifer and Yearly Mean Temp in Europe

In [ ]:
cities = ["London, UK", "Stuttgart, Germany", "Madrid, Spain", "Rome, Italy"]
statistics = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [london.aquifer_temp.head(1).item(), 
                          stuttgart.aquifer_temp.head(1).item(),
                          madrid.aquifer_temp.head(1).item(), 
                          rome.aquifer_temp.head(1).item()],
         'Mean Air Temp': [london.air_temp.mean(), 
                           stuttgart.air_temp.mean(),
                           madrid.air_temp.mean(), 
                           rome.air_temp.mean()],}

x = [(city, stat) for city in cities for stat in statistics]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["london"], color_map["london"], 
                                           color_map["stuttgart"], color_map["stuttgart"],
                                           color_map["madrid"], color_map["madrid"], 
                                           color_map["rome"], color_map["rome"]],
                                   alpha=[0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,]))

aquifer_temp = figure(plot_width=1200, 
                      plot_height=600, 
                      x_range=FactorRange(*x), 
                      title="Comparison of aquifer and mean air temperatures across cities in Europe")
aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
aquifer_temp.y_range.start = 0
aquifer_temp.x_range.range_padding = 0.1
aquifer_temp.xaxis.major_label_orientation = .9
aquifer_temp.xgrid.grid_line_color = None

show(aquifer_temp)

## Thermal Load in USA (Rochester and Fresno) in Winter and Summer

In [ ]:
winter_data = us_data[(us_data.timestamp >= "2010-01-10") & (us_data.timestamp <= "2010-01-24")].copy()
summer_data = us_data[(us_data.timestamp >= "2010-07-15") & (us_data.timestamp <= "2010-07-29")].copy()

winter_data["Total_consumption"] = winter_data["Total_consumption"] / 1000 # To transform consumption into MW
summer_data["Total_consumption"] = summer_data["Total_consumption"] / 1000 # To transform consumption into MW

winter_source = ColumnDataSource(winter_data)
summer_source = ColumnDataSource(summer_data)

winter_min_hour_of_year = winter_data["hourofyear"].min()
winter_max_hour_of_year = winter_data["hourofyear"].max()

summer_min_hour_of_year = summer_data["hourofyear"].min()
summer_max_hour_of_year = summer_data["hourofyear"].max()

winter_min_y_axis_value = winter_data[["Total_consumption", "air_temp"]].min().min() - 5
winter_max_y_axis_value = winter_data[["Total_consumption", "air_temp"]].max().max() + 5

summer_min_y_axis_value = summer_data[["Total_consumption", "air_temp"]].min().min() - 5
summer_max_y_axis_value = summer_data[["Total_consumption", "air_temp"]].max().max() + 5

winter_ticks = [x for x in range(winter_min_hour_of_year, winter_max_hour_of_year + 1, 24)]
summer_ticks = [x for x in range(summer_min_hour_of_year, summer_max_hour_of_year + 1, 24)]

winter_labels_y = [winter_max_y_axis_value for _ in range(15)]
summer_labels_y = [summer_max_y_axis_value for _ in range(15)]

winter_text = ["Su", "Mo", "Tu", "We", "Th", "Fr", "Sa"] * 2 + [""]
summer_text = ["Th", "Fr", "Sa", "Su", "Mo", "Tu", "We",] * 2 + [""]

label_source = ColumnDataSource(dict(winter_x=winter_ticks, winter_y=winter_labels_y, winter_text=winter_text,
                                     summer_x=summer_ticks, summer_y=summer_labels_y, summer_text=summer_text,))



In [ ]:
winter_rochester_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == NOAA2010Dataset.ROCHESTER_NY)])
summer_rochester_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == NOAA2010Dataset.ROCHESTER_NY)])

rochester_thermal_load_winter_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="Rochester, New York, USA - Heat Load from 10/01/2010 to 24/01/2010",
                                            y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                           )
rochester_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
rochester_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
rochester_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks

rochester_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
rochester_thermal_load_winter_2_weeks.xaxis.axis_label = "Hour of the year"
rochester_thermal_load_winter_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_winter = rochester_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_rochester_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = rochester_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_rochester_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

rochester_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rochester_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rochester_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),], location="center"), "below")

rochester_thermal_load_summer_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="Rochester, New York, USA  - Heat Load from 15/07/2010 to 29/07/2010",
                                            y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                           )
rochester_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
rochester_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
rochester_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks

rochester_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
rochester_thermal_load_summer_2_weeks.xaxis.axis_label = "Hour of the year"
rochester_thermal_load_summer_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_summer = rochester_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_rochester_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = rochester_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_rochester_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

rochester_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rochester_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rochester_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),], location="center"), "below")

show(layout([
    [rochester_thermal_load_winter_2_weeks],
    [rochester_thermal_load_summer_2_weeks],
])) 

In [ ]:
winter_fresno_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == NOAA2010Dataset.FRESNO_CA)])
summer_fresno_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == NOAA2010Dataset.FRESNO_CA)])

fresno_thermal_load_winter_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="Fresno, California, USA - Heat Load from 10/01/2010 to 24/01/2010",
                                            y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                           )
fresno_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
fresno_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
fresno_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks

fresno_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
fresno_thermal_load_winter_2_weeks.xaxis.axis_label = "Hour of the year"
fresno_thermal_load_winter_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_winter = fresno_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_fresno_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = fresno_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_fresno_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

fresno_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
fresno_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
fresno_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),], location="center"), "below")

fresno_thermal_load_summer_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="Fresno, California, USA- Heat Load from 15/07/2010 to 29/07/2010",
                                            y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                           )
fresno_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
fresno_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
fresno_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks

fresno_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
fresno_thermal_load_summer_2_weeks.xaxis.axis_label = "Hour of the year"
fresno_thermal_load_summer_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_summer = fresno_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_fresno_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = fresno_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_fresno_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

fresno_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
fresno_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
fresno_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),], location="center"), "below")

show(layout([
    [fresno_thermal_load_winter_2_weeks],
    [fresno_thermal_load_summer_2_weeks],
])) 

## Thermal Load in Europe (London, Rome) in Winter and Summer

In [ ]:
winter_data = europe_data[(europe_data.timestamp >= "2017-01-10") & (europe_data.timestamp <= "2017-01-24")].copy()
summer_data = europe_data[(europe_data.timestamp >= "2017-07-15") & (europe_data.timestamp <= "2017-07-29")].copy()

winter_data["Total_consumption"] = winter_data["Total_consumption"] / 1000 # To transform consumption into MW
summer_data["Total_consumption"] = summer_data["Total_consumption"] / 1000 # To transform consumption into MW

winter_source = ColumnDataSource(winter_data)
summer_source = ColumnDataSource(summer_data)

winter_min_hour_of_year = winter_data["hourofyear"].min()
winter_max_hour_of_year = winter_data["hourofyear"].max()

summer_min_hour_of_year = summer_data["hourofyear"].min()
summer_max_hour_of_year = summer_data["hourofyear"].max()

winter_min_y_axis_value = winter_data[["Total_consumption", "air_temp"]].min().min() - 5
winter_max_y_axis_value = winter_data[["Total_consumption", "air_temp"]].max().max() + 5

summer_min_y_axis_value = summer_data[["Total_consumption", "air_temp"]].min().min() - 5
summer_max_y_axis_value = summer_data[["Total_consumption", "air_temp"]].max().max() + 5

winter_ticks = [x for x in range(winter_min_hour_of_year, winter_max_hour_of_year + 1, 24)]
summer_ticks = [x for x in range(summer_min_hour_of_year, summer_max_hour_of_year + 1, 24)]

winter_labels_y = [winter_max_y_axis_value for _ in range(15)]
summer_labels_y = [summer_max_y_axis_value for _ in range(15)]

winter_text = ["Tu", "We", "Th", "Fr", "Sa", "Su", "Mo"] * 2 + [""]
summer_text = ["Sa", "Su", "Mo", "Tu", "We", "Th", "Fr"] * 2 + [""]

label_source = ColumnDataSource(dict(winter_x=winter_ticks, winter_y=winter_labels_y, winter_text=winter_text,
                                     summer_x=summer_ticks, summer_y=summer_labels_y, summer_text=summer_text,))



In [ ]:
winter_london_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == InsPireDataset.LONDON_UK)])
summer_london_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == InsPireDataset.LONDON_UK)])

london_thermal_load_winter_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="London, United Kingdom - Heat Load from 10/01/2017 to 24/01/2017",
                                            y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                           )
london_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
london_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
london_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks

london_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
london_thermal_load_winter_2_weeks.xaxis.axis_label = "Hour of the year"
london_thermal_load_winter_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_winter = london_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_london_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = london_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_london_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

london_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
london_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
london_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),], location="center"), "below")

london_thermal_load_summer_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="London, United Kingdom - Heat Load from 15/07/2017 to 29/07/2017",
                                            y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                           )
london_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
london_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
london_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks

london_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
london_thermal_load_summer_2_weeks.xaxis.axis_label = "Hour of the year"
london_thermal_load_summer_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_summer = london_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_london_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = london_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_london_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

london_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
london_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
london_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),], location="center"), "below")

show(layout([
    [london_thermal_load_winter_2_weeks],
    [london_thermal_load_summer_2_weeks],
])) 

In [ ]:
winter_rome_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == InsPireDataset.ROME_IT)])
summer_rome_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == InsPireDataset.ROME_IT)])

rome_thermal_load_winter_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="Rome, Italy - Heat Load from 10/01/2017 to 24/01/2017",
                                            y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                           )
rome_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
rome_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
rome_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks

rome_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
rome_thermal_load_winter_2_weeks.xaxis.axis_label = "Hour of the year"
rome_thermal_load_winter_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_winter = rome_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_rome_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = rome_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_rome_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

rome_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rome_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rome_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),], location="center"), "below")

rome_thermal_load_summer_2_weeks = figure(plot_width=1200, 
                                            plot_height=600, 
                                            title="Rome, Italy - Heat Load from 15/07/2017 to 29/07/2017",
                                            y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                           )
rome_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
rome_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
rome_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks

rome_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
rome_thermal_load_summer_2_weeks.xaxis.axis_label = "Hour of the year"
rome_thermal_load_summer_2_weeks.yaxis.axis_label = "Thermal Power [MW]"

l1_summer = rome_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_rome_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = rome_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_rome_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")

rome_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rome_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rome_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),], location="center"), "below")

show(layout([
    [rome_thermal_load_winter_2_weeks],
    [rome_thermal_load_summer_2_weeks],
])) 

## Visualization of thermal load in the USA by season and hour of the day

In [ ]:
hours = range(0,24)
seasons = ["winter", "spring", "summer", "fall"]
cities = [NOAA2010Dataset.MIAMI_FL, NOAA2010Dataset.FRESNO_CA, NOAA2010Dataset.OLYMPIA_WA, NOAA2010Dataset.ROCHESTER_NY]

total_consumption_average_per_hour = dict()
for city in cities:
    total_consumption_average_per_hour[city] = dict()
    
    for season in seasons:
        hours_for_city_and_season = []
        
        for hour in hours:
            hours_for_city_and_season.append(
                us_data[(us_data.city_key == city) & (us_data.season == season) & (us_data.hour == hour)]["Total_consumption"].mean() / 1000
            )
            
        total_consumption_average_per_hour[city][season] = hours_for_city_and_season

x=hours

miami_thermal_load_by_seasons = figure(plot_width=600, 
                           plot_height=300, 
                           title="Miami, FL",
                          )
miami_thermal_load_by_seasons.xaxis.axis_label = "Hour of the day"
miami_thermal_load_by_seasons.yaxis.axis_label = "Thermal load in MW"
miami_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.MIAMI_FL]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
miami_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.MIAMI_FL]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
miami_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.MIAMI_FL]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
miami_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.MIAMI_FL]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)

rochester_thermal_load_by_seasons = figure(plot_width=600, 
                           plot_height=300, 
                           title="Rochester, NY",
                           y_range=miami_thermal_load_by_seasons.y_range,
                           x_range=miami_thermal_load_by_seasons.x_range
                          )
rochester_thermal_load_by_seasons.xaxis.axis_label = "Hour of the day"
rochester_thermal_load_by_seasons.yaxis.axis_label = "Thermal load in MW"
rochester_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.ROCHESTER_NY]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
rochester_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.ROCHESTER_NY]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
rochester_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.ROCHESTER_NY]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
rochester_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.ROCHESTER_NY]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)

olympia_thermal_load_by_seasons = figure(plot_width=600, 
                           plot_height=300, 
                           title="Olympia, WA",
                           y_range=miami_thermal_load_by_seasons.y_range,
                           x_range=miami_thermal_load_by_seasons.x_range
                          )
olympia_thermal_load_by_seasons.xaxis.axis_label = "Hour of the day"
olympia_thermal_load_by_seasons.yaxis.axis_label = "Thermal load in MW"
olympia_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.OLYMPIA_WA]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
olympia_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.OLYMPIA_WA]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
olympia_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.OLYMPIA_WA]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
olympia_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.OLYMPIA_WA]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)

fresno_thermal_load_by_seasons = figure(plot_width=600, 
                           plot_height=300, 
                           title="Fresno, CA",
                           y_range=miami_thermal_load_by_seasons.y_range,
                           x_range=miami_thermal_load_by_seasons.x_range
                          )
fresno_thermal_load_by_seasons.xaxis.axis_label = "Hour of the day"
fresno_thermal_load_by_seasons.yaxis.axis_label = "Thermal load in MW"
fresno_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.FRESNO_CA]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
fresno_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.FRESNO_CA]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
fresno_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.FRESNO_CA]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
fresno_thermal_load_by_seasons.line(x=x, y=total_consumption_average_per_hour[NOAA2010Dataset.FRESNO_CA]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)


show(layout([
    [miami_thermal_load_by_seasons, rochester_thermal_load_by_seasons],
    [olympia_thermal_load_by_seasons, fresno_thermal_load_by_seasons]
])) 

## Visualization of thermal load in Europe by season and hour of the day

In [ ]:
hours = range(0,24)
seasons = ["winter", "spring", "summer", "fall"]
cities = [InsPireDataset.LONDON_UK, InsPireDataset.MADRID_SPA, InsPireDataset.ROME_IT, InsPireDataset.STUTTGART_GER]

total_consumption_average_per_hour = dict()
for city in cities:
    total_consumption_average_per_hour[city] = dict()
    
    for season in seasons:
        hours_for_city_and_season = []
        
        for hour in hours:
            hours_for_city_and_season.append(
                europe_data[(europe_data.city_key == city) & (europe_data.season == season) & (europe_data.hour == hour)]["Total_consumption"].mean() / 1000.00
            )
            
        total_consumption_average_per_hour[city][season] = hours_for_city_and_season

x=hours

london_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title=" London, United Kingdom",
                          )
london_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
london_seasonality_total_consumption.yaxis.axis_label = "Seasonal thermal load [MW]"
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)

stuttgart_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="Stuttgart, Germany",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range
                          )
stuttgart_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
stuttgart_seasonality_total_consumption.yaxis.axis_label = "Seasonal thermal load [MW]"
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)

rome_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="Rome, Italy",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range
                          )
rome_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
rome_seasonality_total_consumption.yaxis.axis_label = "Seasonal thermal load [MW]"
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)

madrid_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title=" Madrid, Spain",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range
                          )
madrid_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
madrid_seasonality_total_consumption.yaxis.axis_label = "Seasonal thermal load [MW]"
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["winter"], legend_label="Winter 2010", color=color_map["winter"], alpha=0.7, line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["spring"], legend_label="Spring 2010", color=color_map["spring"], alpha=0.7,  line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["summer"], legend_label="Summer 2010", color=color_map["summer"], alpha=0.7,  line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["fall"], legend_label="Fall 2010", color=color_map["fall"], alpha=0.7,  line_width=3)


show(layout([
    [london_seasonality_total_consumption, stuttgart_seasonality_total_consumption],
    [rome_seasonality_total_consumption, madrid_seasonality_total_consumption]
])) 

## Heat power signature in the US

The weather dependence of the heat load is shown using a system heat power signature (HPS). A HPS is generated combining the values of thermal demand with the corresponding outdoor temperatures. In this plot we present hourly data, showing a clear linear interdependence between the thermal load and outdoor temperatures for space heating. 

It is expected that solar gains through windows are present between 8 - 15°C, resulting in a smaller heat load than the line representing the HPS. This is a real-world explanation for using the heating degree day threshold temperature when calculating degree-days.

Source: S. Werner, “District Heating and Cooling,” Elsevier, 2013.

In [ ]:
# This calculates the Heating Degree days error using a linear regressor from scipy
rochester_data = us_data[(us_data["city_key"] == NOAA2010Dataset.ROCHESTER_NY) & (us_data["air_temp"] <= 15.0)]
olympia_data = us_data[(us_data["city_key"] == NOAA2010Dataset.OLYMPIA_WA) & (us_data["air_temp"] <= 15.0)]
fresno_data = us_data[(us_data["city_key"] == NOAA2010Dataset.FRESNO_CA) & (us_data["air_temp"] <= 15.0)]
miami_data = us_data[(us_data["city_key"] == NOAA2010Dataset.MIAMI_FL) & (us_data["air_temp"] <= 15.0)]

rochester_slope, rochester_intercept, rochester_r_value, rochester_p_value, rochester_std_err = stats.linregress(rochester_data["air_temp"], rochester_data["Total_consumption"])
rochester_regline = rochester_slope * rochester_data["air_temp"] + rochester_intercept

olympia_slope, olympia_intercept, olympia_r_value, olympia_p_value, olympia_std_err = stats.linregress(olympia_data["air_temp"], olympia_data["Total_consumption"])
olympia_regline = olympia_slope * olympia_data["air_temp"] + olympia_intercept

fresno_slope, fresno_intercept, fresno_r_value, fresno_p_value, fresno_std_err = stats.linregress(fresno_data["air_temp"], fresno_data["Total_consumption"])
fresno_regline = fresno_slope * fresno_data["air_temp"] + fresno_intercept

# Miami does not have any hour with a temperature lower than 15.
# miami_slope, miami_intercept, miami_r_value, miami_p_value, miami_std_err = stats.linregress(miami_data["air_temp"], miami_data["Total_consumption"])
# miami_regline = miami_slope * miami_data["air_temp"] + madrid_intercept


In [ ]:
miami_heat_power_signature = figure(plot_width=600, 
                                    plot_height=300, 
                                    title="Miami, Florida",)
miami_heat_power_signature.yaxis.axis_label = "kW thermal power"
miami_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
miami_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)
# miami_heat_power_signature.line(x=miami_data["air_temp"], y=miami_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


fresno_heat_power_signature = figure(plot_width=600,  
                                     plot_height=300, 
                                     title="Fresno, California",
                                     y_range=miami_heat_power_signature.y_range,
                                     x_range=miami_heat_power_signature.x_range)
fresno_heat_power_signature.yaxis.axis_label = "kW thermal power"
fresno_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
fresno_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
fresno_heat_power_signature.line(x=fresno_data["air_temp"], y=fresno_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


olympia_heat_power_signature = figure(plot_width=600, 
                                      plot_height=300, 
                                      title="Olympia, Washington",
                                      y_range=miami_heat_power_signature.y_range,
                                      x_range=miami_heat_power_signature.x_range)
olympia_heat_power_signature.yaxis.axis_label = "kW thermal power"
olympia_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
olympia_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
olympia_heat_power_signature.line(x=olympia_data["air_temp"], y=olympia_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


rochester_heat_power_signature = figure(plot_width=600, 
                                        plot_height=300, 
                                        title="Rochester, New York",
                                        y_range=miami_heat_power_signature.y_range,
                                        x_range=miami_heat_power_signature.x_range)
rochester_heat_power_signature.yaxis.axis_label = "kW thermal power"
rochester_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
rochester_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
rochester_heat_power_signature.line(x=rochester_data["air_temp"], y=rochester_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)



all_usa_heat_power_signature = figure(plot_width=1200, 
                                      plot_height=300, 
                                      title="Power vs Air Temp",)
all_usa_heat_power_signature.yaxis.axis_label = "Thermal power [kW]"
all_usa_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Fresno, California", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Olympia, Washington", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Rochester, New York", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Miami, Florida", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)


show(layout([
    [all_usa_heat_power_signature],
    [miami_heat_power_signature, fresno_heat_power_signature,],
    [olympia_heat_power_signature, rochester_heat_power_signature],
]))

## Heat power signature in Europe

The weather dependence of the heat load is shown using a system heat power signature (HPS). A HPS is generated combining the values of thermal demand with the corresponding outdoor temperatures. In this plot we present hourly data, showing a clear linear interdependence between the thermal load and outdoor temperatures for space heating. 

It is expected that solar gains through windows are present between 8 - 15°C, resulting in a smaller heat load than the line representing the HPS. This is a real-world explanation for using the heating degree day threshold temperature when calculating degree-days.

Source: S. Werner, “District Heating and Cooling,” Elsevier, 2013.


In [ ]:
# This calculates the Heating Degree days error using a linear regressor from scipy
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK) & (europe_data["air_temp"] <= 15.0)]
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER) & (europe_data["air_temp"] <= 15.0)]
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT) & (europe_data["air_temp"] <= 15.0)]
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA) & (europe_data["air_temp"] <= 15.0)]

london_slope, london_intercept, london_r_value, london_p_value, london_std_err = stats.linregress(london_data["air_temp"], london_data["Total_consumption"])
london_regline = london_slope * london_data["air_temp"] + london_intercept

stuttgart_slope, stuttgart_intercept, stuttgart_r_value, stuttgart_p_value, stuttgart_std_err = stats.linregress(stuttgart_data["air_temp"], stuttgart_data["Total_consumption"])
stuttgart_regline = stuttgart_slope * stuttgart_data["air_temp"] + stuttgart_intercept

rome_slope, rome_intercept, rome_r_value, rome_p_value, rome_std_err = stats.linregress(rome_data["air_temp"], rome_data["Total_consumption"])
rome_regline = rome_slope * rome_data["air_temp"] + rome_intercept

madrid_slope, madrid_intercept, madrid_r_value, madrid_p_value, madrid_std_err = stats.linregress(madrid_data["air_temp"], madrid_data["Total_consumption"])
madrid_regline = madrid_slope * madrid_data["air_temp"] + madrid_intercept


In [ ]:
london_heat_power_signature = figure(plot_width=600, 
                                     plot_height=300, 
                                     title="London, United Kingdom",)
london_heat_power_signature.yaxis.axis_label = "Thermal power [kW]"
london_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
london_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
london_heat_power_signature.line(x=london_data["air_temp"], y=london_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


madrid_heat_power_signature = figure(plot_width=600,  
                                     plot_height=300, 
                                     title="Madrid, Spain",
                                     y_range=london_heat_power_signature.y_range,
                                     x_range=london_heat_power_signature.x_range)
madrid_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
madrid_heat_power_signature.yaxis.axis_label = "Thermal power [kW]"
madrid_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
madrid_heat_power_signature.line(x=madrid_data["air_temp"], y=madrid_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


rome_heat_power_signature = figure(plot_width=600, 
                                   plot_height=300, 
                                   title="Rome, Italy",
                                   y_range=london_heat_power_signature.y_range,
                                   x_range=london_heat_power_signature.x_range)
rome_heat_power_signature.yaxis.axis_label = "Thermal power [kW]"
rome_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
rome_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
rome_heat_power_signature.line(x=rome_data["air_temp"], y=rome_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


stuttgart_heat_power_signature = figure(plot_width=600, 
                                        plot_height=300, 
                                        title="Stuttgart, Germany",
                                        y_range=london_heat_power_signature.y_range,
                                        x_range=london_heat_power_signature.x_range)
stuttgart_heat_power_signature.yaxis.axis_label = "Thermal power [kW]"
stuttgart_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
stuttgart_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
stuttgart_heat_power_signature.line(x=stuttgart_data["air_temp"], y=stuttgart_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


all_europe_heat_power_signature = figure(plot_width=1200, 
                                         plot_height=300, 
                                         title="Power vs Air Temp",)
all_europe_heat_power_signature.yaxis.axis_label = "Thermal power [kW]"
all_europe_heat_power_signature.xaxis.axis_label = "Outdoor temperature"
all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="London, UK", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)



show(layout([
    [all_europe_heat_power_signature],
    [london_heat_power_signature, stuttgart_heat_power_signature],
    [rome_heat_power_signature, madrid_heat_power_signature],

]))

## Load Duration Curve for Europe

In a heat load duration (HLD) diagram the hourly or daily heat loads are sorted in descending order, with the highest thermal load of the year (winter peaks) located to the left and the lowest to the right (summer nights). The HLD diagram will be different annualy depending on the weather conditions, therefore for planning purposes it is recommended to use long-term duration diagrams based on historical data.

Source: S. Werner, “District Heating and Cooling,” Elsevier, 2013.

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["Total_consumption"]/1000
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["Total_consumption"]/1000
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["Total_consumption"]/1000
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["Total_consumption"]/1000

x = np.array(range(1,london_data.shape[0] + 1))
x_max = max(x)
x_cumsum = x * 100 / x_max 

y_london = london_data.sort_values(ascending=False)
y_london_max = london_data.max()
y_london_norm = y_london * 100 / y_london_max


y_stuttgart = stuttgart_data.sort_values(ascending=False)
y_stuttgart_max = stuttgart_data.max()
y_stuttgart_norm = y_stuttgart * 100 / y_stuttgart_max


y_rome = rome_data.sort_values(ascending=False)
y_rome_max = rome_data.max()
y_rome_norm = y_rome * 100 / y_rome_max


y_madrid = madrid_data.sort_values(ascending=False)
y_madrid_max = madrid_data.max()
y_madrid_norm = y_madrid * 100 / y_madrid_max

max_across_europe = max(y_london_max, y_stuttgart_max, y_rome_max, y_madrid_max,)
y_london_norm_all =  y_london * 100 / max_across_europe
y_stuttgart_norm_all =  y_stuttgart * 100 / max_across_europe
y_rome_norm_all =  y_rome * 100 / max_across_europe
y_madrid_norm_all =  y_madrid * 100 / max_across_europe

In [ ]:


load_duration_curve_europe_y_equal_x_equal = figure(plot_width=600, 
                                plot_height=300, 
                                title="LDC for Europe",
                                x_range=(0,9000))
load_duration_curve_europe_y_equal_x_equal.yaxis.axis_label = "Thermal power [MW]"
load_duration_curve_europe_y_equal_x_equal.xaxis.axis_label = "Duration, hours per year"
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_london, legend_label="London ", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_stuttgart, legend_label="Stuttgart ", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_rome, legend_label="Rome ", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_madrid, legend_label="Madrid ", color=color_map["madrid"], alpha=1, line_width=2.5)

load_duration_curve_europe_y_equal_x_norm = figure(plot_width=600, 
                                plot_height=300, 
                                title="LDC for Europe",
                                x_range=(0,105))
load_duration_curve_europe_y_equal_x_norm.yaxis.axis_label = "Thermal power [MW]"
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_london, legend_label="London ", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_stuttgart, legend_label="Stuttgart ", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_rome, legend_label="Rome ", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_madrid, legend_label="Madrid ", color=color_map["madrid"], alpha=1, line_width=2.5)


load_duration_curve_europe_y_norm_x_equal = figure(plot_width=600, 
                                plot_height=300,
                                title="LDC for Europe",
                                x_range=(0,9000))
load_duration_curve_europe_y_norm_x_equal.xaxis.axis_label = "Duration, hours per year"
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_london_norm_all, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_stuttgart_norm_all, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_rome_norm_all, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_madrid_norm_all, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


load_duration_curve_europe_y_norm_x_norm = figure(plot_width=600, 
                                plot_height=300,
                                title="LDC for Europe",
                                x_range=(0,105))
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_london_norm_all, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_stuttgart_norm_all, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_rome_norm_all, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_madrid_norm_all, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


show(layout([
    [load_duration_curve_europe_y_equal_x_equal, load_duration_curve_europe_y_equal_x_norm],
    [load_duration_curve_europe_y_norm_x_equal, load_duration_curve_europe_y_norm_x_norm],
]))



In [ ]:
load_duration_curve_europe_norm_y_norm_x_equal = figure(plot_width=1200, 
                                plot_height=600, 
                                title="LDC for Europe",
                                x_range=(0,9000))
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


load_duration_curve_europe_norm_y_norm_x_norm = figure(plot_width=1200, 
                                plot_height=600, 
                                title="LDC for Europe",
                                x_range=(0,105))
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)



#load_duration_curve_europe_norm = figure(plot_width=1200, 
#                                plot_height=600, 
#                                title="LDC for Europe",
 #                               x_range=(0,9000))
#load_duration_curve_europe_norm.line(x=x, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)



#load_duration_curve_europe_norm = figure(plot_width=1200, 
#                                plot_height=600, 
#                                title="LDC for Europe",
#                                x_range=(0,105))
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


show(layout([
    [load_duration_curve_europe_norm_y_norm_x_equal],[load_duration_curve_europe_norm_y_norm_x_norm],
    #[load_duration_curve_europe_y_norm_x_equal, load_duration_curve_europe_y_norm_x_norm],
]))



In [ ]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.models import Legend


def make_plot(title, hist, edges, x, pdf=None, cdf=None):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, 
           bottom=0, 
           left=edges[:-1], 
           right=edges[1:],
           fill_color="navy", 
           line_color="white", 
           alpha=0.5)
    p.y_range.start = 0
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    #p.grid.grid_line_color="white"
    return p

# Normal Distribution

#mu, sigma = 0, 0.5

#measured = np.random.normal(mu, sigma, 1000)
#hist, edges = np.histogram(measured, density=True, bins=50)

measured = london_data.sort_values(ascending=False)
hist, edges = np.histogram(measured, density=False, bins=50)

#x = np.linspace(-2, 2, 1000)
#pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
#cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1 = make_plot("Normal Distribution (μ=0, σ=0.5)", hist, edges, x)
show(layout([
    [p1]
]))

## Sources and network temperatures (°C)

In [ ]:
dates = pd.date_range('1/1/2017', periods=168)

source_europe = ColumnDataSource({"timestamp": dates,
                                 
                                 "london_source1_temp": london.source1_temp,
                                 "london_source2_temp": london.source2_temp,
                                 "london_aquifer_temp": london.aquifer_temp,
                                  "london_network_temp": london.net_temp,
                                })


london_sources_temp = figure(plot_width=1200, 
                                plot_height=400, 
                                x_axis_type="datetime",)
                                
a= london_sources_temp.line(x="timestamp", y="london_source1_temp", color='navy', alpha=0.5, source=source_europe, line_width=2)
b= london_sources_temp.line(x="timestamp", y="london_source2_temp",  color='green', alpha=0.5, source=source_europe, line_width=2)
c= london_sources_temp.line(x="timestamp", y="london_aquifer_temp",  color='red', alpha=0.5, source=source_europe, line_width=2)
d= london_sources_temp.triangle(x="timestamp", y="london_network_temp", color='orange', alpha=0.5, source=source_europe, size=5)

legend = Legend(items=[
    ("Source 1"   , [a]),
    ("Source 2" , [b]),
    ("Ground source" , [c]),
    ("Network" , [d]),
], location="center")

london_sources_temp.add_layout(legend, 'right')
london_sources_temp.xaxis.axis_label = "Jan 1 to 7"
london_sources_temp.yaxis.axis_label = "Temperature (°C)"

show(london_sources_temp)

In [ ]:
dates = pd.date_range('1/10/2017', periods=350, freq ='H')

source_europe = ColumnDataSource({"timestamp": dates,
                                 
                                 "london_user_temp": london.user_temp,
                                
                                })

london_users_temp = figure(plot_width=1200, 
                                plot_height=400, 
                                x_axis_type="datetime",
                                title ="Temperature delivered to buildings")
                                
a= london_users_temp.line(x="timestamp", y="london_user_temp", color='navy', alpha=0.5, source=source_europe, line_width=2)


#legend = Legend(items=[
#    (""   , [a]),
#], location="center")

#london_users_temp.add_layout(legend, 'right')
london_users_temp.xaxis.axis_label = "Jan 10 to 25"
london_users_temp.yaxis.axis_label = "Temperature (°C)"

show(london_users_temp)

## Heat pumps electricity consumption by city

In [ ]:
us_data

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["E_el"]/1000
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["E_el"]/1000
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["E_el"]/1000
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["E_el"]/1000

miami_data= us_data[(us_data["city_key"] == NOAA2010Dataset.MIAMI_FL)]["E_el"]/1000
fresno_data= us_data[(us_data["city_key"] == NOAA2010Dataset.FRESNO_CA)]["E_el"]/1000
olympia_data= us_data[(us_data["city_key"] == NOAA2010Dataset.OLYMPIA_WA)]["E_el"]/1000
rochester_data= us_data[(us_data["city_key"] == NOAA2010Dataset.ROCHESTER_NY)]["E_el"]/1000



x = ["London, UK", "Madrid, SPA", "Rome, IT", "Stuttgart, GER", "Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
top = [london_data.sum(), madrid_data.sum(), rome_data.sum(), stuttgart_data.sum(), miami_data.sum(), fresno_data.sum(), olympia_data.sum(), rochester_data.sum()]

europe_electricity = figure(plot_width=1200, 
                                plot_height=300, 
                                x_range= FactorRange(*x),
                                title="Annual consumption by country",)

europe_electricity.yaxis.axis_label = "Electric power [MW]"
europe_electricity.vbar(x=x, top=top, width=0.9,alpha=0.8)



show(layout([
    [europe_electricity],
]))


In [ ]:
miami.columns

## Heat losses by country

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["E_loss_tot"]/1000
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["E_loss_tot"]/1000
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["E_loss_tot"]/1000
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["E_loss_tot"]/1000

miami_data= us_data[(us_data["city_key"] == NOAA2010Dataset.MIAMI_FL)]["E_loss_tot"]/1000
fresno_data= us_data[(us_data["city_key"] == NOAA2010Dataset.FRESNO_CA)]["E_loss_tot"]/1000
olympia_data= us_data[(us_data["city_key"] == NOAA2010Dataset.OLYMPIA_WA)]["E_loss_tot"]/1000
rochester_data= us_data[(us_data["city_key"] == NOAA2010Dataset.ROCHESTER_NY)]["E_loss_tot"]/1000

        
x = ["London, UK", "Madrid, SPA", "Rome, IT", "Stuttgart, GER", "Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
top = [london_data.sum(), madrid_data.sum(), rome_data.sum(), stuttgart_data.sum(), miami_data.sum(), fresno_data.sum(), olympia_data.sum(), rochester_data.sum()]

europe_heat_loss = figure(plot_width=1200, 
                                plot_height=400, 
                                x_range= FactorRange(*x),
                                title="Total heat loss by country, Europe",)
europe_heat_loss.vbar(x=x, top=top, width=0.9,alpha=0.8)
europe_heat_loss.yaxis.axis_label = "Thermal energy [GWh]"


show(layout([
    [europe_heat_loss],
]))

## Visualization of Hourly Thermal Power Use for each season and hour of day

## Visualization of Hourly Electrical Power Use for each season and hour of day: Europe

In [ ]:
hours = range(0,24)
seasons = ["winter", "spring", "summer", "fall"]
cities = [InsPireDataset.LONDON_UK, InsPireDataset.MADRID_SPA, InsPireDataset.ROME_IT, InsPireDataset.STUTTGART_GER]

total_consumption_average_per_hour = dict()
for city in cities:
    total_consumption_average_per_hour[city] = dict()
    
    for season in seasons:
        hours_for_city_and_season = []
        
        for hour in hours:
            hours_for_city_and_season.append(
                europe_data[(europe_data.city_key == city) & (europe_data.season == season) & (europe_data.hour == hour)]["E_el"].mean() / 1000
            )
            
        total_consumption_average_per_hour[city][season] = hours_for_city_and_season

x=hours

london_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="London, United Kingdom",
                          )
london_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
london_seasonality_total_consumption.yaxis.axis_label = "Seasonal electric power [MW]"
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)

stuttgart_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="Stuttgart, Germany",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range
                          )
stuttgart_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
stuttgart_seasonality_total_consumption.yaxis.axis_label = "Seasonal electric power [MW]"
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)

rome_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="Rome, Italy",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range
                          )
rome_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
rome_seasonality_total_consumption.yaxis.axis_label = "Seasonal electric power [MW]"
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)

madrid_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="Madrid, Spain",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range
                          )
madrid_seasonality_total_consumption.xaxis.axis_label = "Hour of the day"
madrid_seasonality_total_consumption.yaxis.axis_label = "Seasonal electric power [MW]"
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)


show(layout([
    [london_seasonality_total_consumption, stuttgart_seasonality_total_consumption],
    [rome_seasonality_total_consumption, madrid_seasonality_total_consumption]
])) 

## Visualization of Hourly Electrical Power Use for each season and day of the week: Europe

In [ ]:
london

In [ ]:
dayofweek = range(0,7)
seasons = ["winter", "spring", "summer", "fall"]
cities = [InsPireDataset.LONDON_UK, InsPireDataset.MADRID_SPA, InsPireDataset.ROME_IT, InsPireDataset.STUTTGART_GER]

total_consumption_average_per_hour = dict()
for city in cities:
    total_consumption_average_per_hour[city] = dict()
    
    for season in seasons:
        days_for_city_and_season = []
        
        for day in dayofweek:
            days_for_city_and_season.append(
                europe_data[(europe_data.city_key == city) & (europe_data.season == season) & (europe_data.dayofweek == day)]["E_el"].mean()/1000
            )
            
        total_consumption_average_per_hour[city][season] = days_for_city_and_season

x=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

london_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="London, United Kingdom",
                                              x_range=FactorRange(*x),
                                              x_axis_label="Day of the week",
                                                 y_axis_label="Seasonal electric power [MW]",
                          )
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
london_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.LONDON_UK]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)

stuttgart_seasonality_total_consumption = figure(plot_width=600, 
                                                 plot_height=300, 
                                                 title="Stuttgart, Germany",
                                                 y_range=london_seasonality_total_consumption.y_range,
                                                 x_range=london_seasonality_total_consumption.x_range,
                                                 x_axis_label="Day of the week",
                                                 y_axis_label="Seasonal electric power [MW]",
                                                )
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
stuttgart_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.STUTTGART_GER]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)

rome_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="Rome, Italy",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range,
                                            x_axis_label="Day of the week",
                                                 y_axis_label="Seasonal electric power [MW]",
                          )
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
rome_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.ROME_IT]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)

madrid_seasonality_total_consumption = figure(plot_width=600, 
                           plot_height=300, 
                           title="Madrid, Spain",
                           y_range=london_seasonality_total_consumption.y_range,
                           x_range=london_seasonality_total_consumption.x_range,
                                              x_axis_label="Day of the week",
                                                 y_axis_label="Power in MW",
                          )
madrid_seasonality_total_consumption.xaxis.axis_label = "Day of the week"
madrid_seasonality_total_consumption.yaxis.axis_label = "Seasonal electric power [MW]"
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["winter"], legend_label="Winter 2010", color=color_map["london"], alpha=0.7, line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["spring"], legend_label="Spring 2010", color=color_map["stuttgart"], alpha=0.7,  line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["summer"], legend_label="Summer 2010", color=color_map["rome"], alpha=0.7,  line_width=3)
madrid_seasonality_total_consumption.line(x=x, y=total_consumption_average_per_hour[InsPireDataset.MADRID_SPA]["fall"], legend_label="Fall 2010", color=color_map["madrid"], alpha=0.7,  line_width=3)


show(layout([
    [london_seasonality_total_consumption, stuttgart_seasonality_total_consumption],
    [rome_seasonality_total_consumption, madrid_seasonality_total_consumption]
])) 

## Visualizing supply of heat during a year for each location

In [ ]:
# I set the index as a time index in order to visualize the daily behavior
#europe_data = europe_data.set_index('timestamp')
stuttgart = stuttgart.set_index('timestamp')
london = london.set_index('timestamp')
rome = rome.set_index('timestamp')
madrid = madrid.set_index('timestamp')

In [ ]:
columns = ['city_key','Total_consumption', 'E_el']

london_daily_data = london[columns].resample('D').mean()
madrid_daily_data = madrid[columns].resample('D').mean()
stuttgart_daily_data = stuttgart[columns].resample('D').mean()
rome_daily_data = rome[columns].resample('D').mean()

days = range(1,366)

london_daily_data = figure(plot_width=600, 
                                plot_height=300,
                                title="London, United Kingdom",
                            
london_daily_data.line(x=days, y=london_daily_data['Total_consumption'], legend_label="Thermal demand", color=color_map["london"])
london_daily_data.line(x=days, y=london_daily_data['E_el'], legend_label="Electric power", color=color_map["london"], source=london_daily_data)

#madrid_daily_data = figure(plot_width=600,  
                                # plot_height=300, 
                               # title="Madrid, Spain",
                              

#madrid_daily_data.line(x=days, y="Total_consumption", legend_label="Thermal demand", color=color_map["madrid"], alpha=0.2, source=madrid_daily_data)
#madrid_daily_data.circle(x="timestamp", y="E_el", legend_label="Electric power", color=color_map["madrid"], alpha=0.4, source=madrid_daily_data, size=1)

#rome_daily_data = figure(plot_width=600, 
                                  #plot_height=300, 
                                #title="Rome, Italy",
                              
#rome_daily_data.line(x=days, y="Total_consumption", legend_label="Thermal demand", color=color_map["rome"], alpha=0.2, source=rome_daily_data view=rome_view)
#rome_daily_data.circle(x="timestamp", y="E_el", legend_label="Electric power", color=color_map["rome"], alpha=0.4, source=rome_daily_data, view=rome_view, size=1)

#stuttgart_daily_data = figure(plot_width=600, 
                                    #plot_height=300, 
                                #title="Stuttgart, Germany",
                               
#stuttgart_daily_data.line(x=days, y="Total_consumption", legend_label="Thermal demand", color=color_map["stuttgart"], alpha=0.2, source=stuttgart_daily_data)
#stuttgart_daily_data.circle(x="timestamp", y="E_el", legend_label="Electric power", color=color_map["stuttgart"], alpha=0.4, source=stuttgart_daily_data, size=1)

                           
show(layout([
    [london_daily_data, stuttgart_daily_data ,],
    [rome_daily_data, madrid_daily_data],
]))

In [ ]:
print(madrid_daily_data)

## Heatmap

When doing heatmaps with Bokeh using the dates as x-axis and hours of day in the y-axis, we have to include only the date part of the timestamps. This is because we are including the hour in the y-axis, if we don't do this, bokeh will interpret the following points (2010-01-01 15:00, 13, 10000) and (2010-01-01 14:00, 14, 5000) in different positions with respect to the x-axis. Instead, if we supply the values as (2010-01-01, 13, 10000) and (2010-01-01, 14, 5000), then these two will be positioned in the same x-axis point.

### Good example

![Good Example](images/heatmap_good.png)

### Bad Example

![Bad Example](images/heatmap_bad.png)

Another thing to take into account is that the width of `rect` plots with `datetime` x-axis is equal to $1ms$, i.e., if we want the rectangle to be of width of a full day, we have to provide a width equivalent of 1 day in miliseconds, e.g. $width = 1 * 24 * 60 * 60 * 1000 = 86.400.000$

In [ ]:
# The width on these plots is equivalent to ms in datetime, i.e., width = 1 === width = 1ms
ONE_DAY_WIDTH = 1 * 24 * 60 * 60 * 1000

mapper = LinearColorMapper(palette=Spectral11, low=europe_data["Total_consumption"].min(), high=europe_data["Total_consumption"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

london_total_consumption_heatmap = figure(plot_width=1200, plot_height=600,  x_axis_label="Days", y_axis_label="Hour", x_axis_type="datetime", )
london_total_consumption_heatmap.grid.visible = False
london_total_consumption_heatmap.axis.axis_line_color = None
london_total_consumption_heatmap.axis.major_tick_line_color = None
london_total_consumption_heatmap.axis.major_label_text_font_size = "7px"
london_total_consumption_heatmap.axis.major_label_standoff = 0
london_total_consumption_heatmap.xaxis.major_label_orientation = 1.0

madrid_total_consumption_heatmap = figure(plot_width=1200, plot_height=600,  x_axis_label="Days", y_axis_label="Hour", x_axis_type="datetime", )
madrid_total_consumption_heatmap.grid.visible = False
madrid_total_consumption_heatmap.axis.axis_line_color = None
madrid_total_consumption_heatmap.axis.major_tick_line_color = None
madrid_total_consumption_heatmap.axis.major_label_text_font_size = "7px"
madrid_total_consumption_heatmap.axis.major_label_standoff = 0
madrid_total_consumption_heatmap.xaxis.major_label_orientation = 1.0

rome_total_consumption_heatmap = figure(plot_width=1200, plot_height=600, x_axis_label="Days", y_axis_label="Hour", x_axis_type="datetime", )
rome_total_consumption_heatmap.grid.visible = False
rome_total_consumption_heatmap.axis.axis_line_color = None
rome_total_consumption_heatmap.axis.major_tick_line_color = None
rome_total_consumption_heatmap.axis.major_label_text_font_size = "7px"
rome_total_consumption_heatmap.axis.major_label_standoff = 0
rome_total_consumption_heatmap.xaxis.major_label_orientation = 1.0

stuttgart_total_consumption_heatmap = figure(plot_width=1200, plot_height=600,  x_axis_label="Days", y_axis_label="Hour", x_axis_type="datetime", )
stuttgart_total_consumption_heatmap.grid.visible = False
stuttgart_total_consumption_heatmap.axis.axis_line_color = None
stuttgart_total_consumption_heatmap.axis.major_tick_line_color = None
stuttgart_total_consumption_heatmap.axis.major_label_text_font_size = "7px"
stuttgart_total_consumption_heatmap.axis.major_label_standoff = 0
stuttgart_total_consumption_heatmap.xaxis.major_label_orientation = 1.0

london_total_consumption_heatmap.rect(x="date", 
                 y="hour", 
                 width=ONE_DAY_WIDTH, 
                 height=1, 
                 source=europe_source,
                 view=london_view,
                 line_color=None, 
                 fill_color=transform('Total_consumption', mapper))
london_total_consumption_heatmap.add_layout(color_bar, 'right')

madrid_total_consumption_heatmap.rect(x="date", 
                 y="hour", 
                 width=ONE_DAY_WIDTH, 
                 height=1, 
                 source=europe_source,
                 view=madrid_view,
                 line_color=None, 
                 fill_color=transform('Total_consumption', mapper))
madrid_total_consumption_heatmap.add_layout(color_bar, 'right')

rome_total_consumption_heatmap.rect(x="date", 
                 y="hour", 
                 width=ONE_DAY_WIDTH, 
                 height=1, 
                 source=europe_source,
                 view=rome_view,
                 line_color=None, 
                 fill_color=transform('Total_consumption', mapper))
rome_total_consumption_heatmap.add_layout(color_bar, 'right')

stuttgart_total_consumption_heatmap.rect(x="date", 
                 y="hour", 
                 width=ONE_DAY_WIDTH, 
                 height=1, 
                 source=europe_source,
                 view=stuttgart_view,
                 line_color=None, 
                 fill_color=transform('Total_consumption', mapper))
stuttgart_total_consumption_heatmap.add_layout(color_bar, 'right')

show(layout([
    [london_total_consumption_heatmap],
    [stuttgart_total_consumption_heatmap],
    [rome_total_consumption_heatmap],
    [madrid_total_consumption_heatmap],
]))

In [ ]:
london.columns

In [ ]:
data = london.copy()

data_month = data.groupby(by=["month"]).sum()/1e6 
source_month = ColumnDataSource(data_month)

data_week = data.groupby(by=["dayofweek"]).mean() #/ 1000
source_week = ColumnDataSource(data_week)

data_week_winter = data[data.season == "winter"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_winter = ColumnDataSource(data_week_winter)

data_week_spring = data[data.season == "spring"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_spring = ColumnDataSource(data_week_spring)

data_week_summer = data[data.season == "summer"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_summer = ColumnDataSource(data_week_summer)

data_week_fall = data[data.season == "fall"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_fall = ColumnDataSource(data_week_fall)

london_stack_month = figure(plot_width=1200, plot_height=300, title ="London, UK")
london_stack_month.yaxis.axis_label = "Monthly thermal energy [GWh]"
london_stack_month.xaxis.axis_label = "Month"
london_stack_month.varea_stack(x='month', stackers=['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"], fill_color=["#b33693", "#66dd87", "#ccd9ff", "#ffff0f"], legend_label = ["Source 1","Source 2","Aquifer wells","Electricity"], source=source_month)

london_stack_dayofweek_winter = figure(plot_width=600, plot_height=300, title ="Winter season")
london_stack_dayofweek_winter.yaxis.axis_label = "Average thermal energy [kWh]"
london_stack_dayofweek_winter.xaxis.axis_label = "Day of week"
london_stack_dayofweek_winter.varea_stack(x='dayofweek', stackers=['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"], fill_color=["#b33693", "#66dd87", "#ccd9ff", "#ffff0f"], source=source_week_winter)

london_stack_dayofweek_spring = figure(plot_width=600, plot_height=300, title ="Spring season",
                                       y_range=london_stack_dayofweek_winter.y_range,
                                       x_range=london_stack_dayofweek_winter.x_range)
london_stack_dayofweek_spring.varea_stack(x='dayofweek', stackers=['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"], fill_color=["#b33693", "#66dd87", "#ccd9ff", "#ffff0f"], source=source_week_spring)
london_stack_dayofweek_spring.yaxis.axis_label = "Average thermal energy [kWh]"
london_stack_dayofweek_spring.xaxis.axis_label = "Day of week"

london_stack_dayofweek_summer = figure(plot_width=600, plot_height=300, title ="Summer season",
                                       y_range=london_stack_dayofweek_winter.y_range,
                                       x_range=london_stack_dayofweek_winter.x_range)
london_stack_dayofweek_summer.varea_stack(x='dayofweek', stackers=['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"], fill_color=["#b33693", "#66dd87", "#ccd9ff", "#ffff0f"], source=source_week_summer)
london_stack_dayofweek_summer.yaxis.axis_label = "Average thermal energy [kWh]"
london_stack_dayofweek_summer.xaxis.axis_label = "Day of week"

london_stack_dayofweek_fall = figure(plot_width=600, plot_height=300, title ="Fall season",
                                     y_range=london_stack_dayofweek_winter.y_range,
                                     x_range=london_stack_dayofweek_winter.x_range)
london_stack_dayofweek_fall.varea_stack(x='dayofweek', stackers=['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"], fill_color=["#b33693", "#66dd87", "#ccd9ff", "#ffff0f"], source=source_week_fall)
london_stack_dayofweek_fall.yaxis.axis_label = "Average thermal energy [kWh]"
london_stack_dayofweek_fall.xaxis.axis_label = "Day of week"

london_stack_dayofweek = figure(plot_width=1200, plot_height=600, title ="Average thermal energy by day of week")
london_stack_dayofweek.varea_stack(x='dayofweek', stackers=['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"], fill_color=["#b33693", "#66dd87", "#ccd9ff", "#ffff0f"], source=source_week)
london_stack_dayofweek.yaxis.axis_label = "Average thermal energy [kWh]"
london_stack_dayofweek.xaxis.axis_label = "Day of week"


show(layout([
    [london_stack_month],
    [london_stack_dayofweek_winter, london_stack_dayofweek_spring],
    [london_stack_dayofweek_summer, london_stack_dayofweek_fall],
    [london_stack_dayofweek],
]))

In [ ]:
import pandas as pd

from bokeh.io import output_file, show
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter,)
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import transform

print(f"{data=}")

data.Year = data.Year.astype(str)
data = data.set_index('Year')
data.drop('Annual', axis=1, inplace=True)
data.columns.name = 'Month'


# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()
print(f"{df=}")

source = ColumnDataSource(df)

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette="Viridis256", low=df.rate.min(), high=df.rate.max())

p = figure(plot_width=800, plot_height=300, title="US Unemployment 1948—2016",
           x_range=list(data.index), y_range=list(reversed(data.columns)),
           toolbar_location=None, tools="", x_axis_location="above")

p.rect(x="Year", y="Month", width=1, height=1, source=source,
       line_color=None, fill_color=transform('rate', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"))

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

# Y -> Total_consumption
# X -> Mes

show(p)

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy import stats

#london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK) & 
                         # (europe_data["air_temp"] <= 15.0)]
#stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER) & 
                          #(europe_data["air_temp"] <= 15.0)]
#rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT) & 
                          #(europe_data["air_temp"] <= 15.0)]
#madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA) & 
                          #(europe_data["air_temp"] <= 15.0)]

#london_slope, london_intercept, london_r_value, london_p_value, london_std_err = stats.linregress(london_data["air_temp"], 
                                                                                                  #london_data["Total_consumption"])
#london_regline = london_slope * london_data["air_temp"] + london_intercept

#stuttgart_slope, stuttgart_intercept, stuttgart_r_value, stuttgart_p_value, stuttgart_std_err = stats.linregress(stuttgart_data["air_temp"], 
                                                                                                  #stuttgart_data["Total_consumption"])
#stuttgart_regline = stuttgart_slope * stuttgart_data["air_temp"] + stuttgart_intercept

#rome_slope, rome_intercept, rome_r_value, rome_p_value, rome_std_err = stats.linregress(rome_data["air_temp"], 
                                                                                                  #rome_data["Total_consumption"])
#rome_regline = rome_slope * rome_data["air_temp"] + rome_intercept

#madrid_slope, madrid_intercept, madrid_r_value, madrid_p_value, madrid_std_err = stats.linregress(madrid_data["air_temp"], 
                                                                                                 #a madrid_data["Total_consumption"])
#madrid_regline = madrid_slope * madrid_data["air_temp"] + madrid_intercept



london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                title="London, United Kingdom",)
london_air_temp_and_fit.circle(x="air_temp", y="E_loss_tot", legend_label="Heat losses vs outdoor temperature", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
#london_air_temp_and_fit.line(x=london_data["air_temp"], y=london_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.circle(x="air_temp", y="E_loss_tot", legend_label="Heat losses vs outdoor temperature", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
#madrid_air_temp_and_fit.line(x=madrid_data["air_temp"], y=madrid_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.circle(x="air_temp", y="E_loss_tot", legend_label="Heat losses vs outdoor temperature", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
#rome_air_temp_and_fit.line(x=rome_data["air_temp"], y=rome_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.circle(x="air_temp", y="E_loss_tot", legend_label="Heat losses vs outdoor temperature", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
#stuttgart_air_temp_and_fit.line(x=stuttgart_data["air_temp"], y=stuttgart_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


all_europe = figure(plot_width=1200, 
                    plot_height=300, 
                    title="Power vs Air Temp",)
all_europe.circle(x="air_temp", y="E_loss_tot", legend_label="London, UK", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
all_europe.circle(x="air_temp", y="E_loss_tot", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
all_europe.circle(x="air_temp", y="E_loss_tot", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
all_europe.circle(x="air_temp", y="E_loss_tot", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)



show(layout([
    [all_europe],
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],

]))

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["E_el"]
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["E_el"]
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["E_el"]
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["E_el"]

x = np.array(range(1,london_data.shape[0] + 1))

london_electricity = figure(plot_width=600, 
                                plot_height=300, 
                                title="London, United Kingdom",)
london_electricity.vbar(x=x, y=london_data.sum(), legend_label="Air Temp", color=color_map["london"], alpha=0.2)


madrid_electricity = figure(plot_width=600,  
                                 plot_height=300, 
                                title="Madrid, Spain",)
madrid_electricity.vbar(x=x, y=madrid_data.sum(), legend_label="Air Temp", color=color_map["madrid"], alpha=0.2)

rome_electricity = figure(plot_width=600, 
                                  plot_height=300, 
                                title="Rome, Italy",)
rome_electricity.vbar(x=x, y=rome_data.sum(), legend_label="Air Temp", color=color_map["rome"], alpha=0.2)

stuttgart_electricity = figure(plot_width=600, 
                                    plot_height=300, 
                                title="Stuttgart, Germany",)
stuttgart_electricity.vbar(x=x, y=stuttgart_data.sum(), legend_label="Air Temp", color=color_map["stuttgart"], alpha=0.2)


show(layout([
    [london_electricity, madrid_electricity],
    [rome_electricity, stuttgart_electricity],
]))


### Annual analysis 

In [ ]:
thermal_load_y=[london['Total_consumption'].sum(), 
                stuttgart['Total_consumption'].sum(), 
                madrid['Total_consumption'].sum(), 
                rome['Total_consumption'].sum(), 
                miami['Total_consumption'].sum(), 
                fresno['Total_consumption'].sum(), 
                olympia['Total_consumption'].sum(), 
                rochester['Total_consumption'].sum()]
thermal_y =pd.Series(thermal_load_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
thermal_y

electric_cons_y=[london['E_el'].sum(), 
                 stuttgart['E_el'].sum(), 
                 madrid['E_el'].sum(),
                 rome['E_el'].sum(), 
                 miami['E_el'].sum(), 
                 fresno['E_el'].sum(), 
                 olympia['E_el'].sum(), 
                 rochester['E_el'].sum()]
electric_y =pd.Series(electric_cons_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
electric_y

heat_loss_y=[london['E_loss_tot'].sum(), 
             stuttgart['E_loss_tot'].sum(), 
             madrid['E_loss_tot'].sum(),
             rome['E_loss_tot'].sum(), 
             miami['E_loss_tot'].sum(), 
             fresno['E_loss_tot'].sum(), 
             olympia['E_loss_tot'].sum(), 
             rochester['E_loss_tot'].sum()]
heat_loss_y =pd.Series(heat_loss_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
heat_loss_y


thermal_energy_network= thermal_y - electric_y
#thermal_energy_network =pd.Series(thermal_energy_network)

thermal_peak=[london['Total_consumption'].max()/1000, 
              stuttgart['Total_consumption'].max()/1000, 
              madrid['Total_consumption'].max()/1000, 
              rome['Total_consumption'].max()/1000, 
              miami['Total_consumption'].max()/1000, 
              fresno['Total_consumption'].max()/1000, 
              olympia['Total_consumption'].max()/1000, 
              rochester['Total_consumption'].max()/1000]
thermal_peak_MW=pd.Series(thermal_peak, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])


COP = thermal_y / electric_y

source1_thermal_y= [london['heat_source1'].sum(), 
                    stuttgart['heat_source1'].sum(), 
                    madrid['heat_source1'].sum(), 
                    rome['heat_source1'].sum(), 
                    miami['heat_source1'].sum(), 
                    fresno['heat_source1'].sum(), 
                    olympia['heat_source1'].sum(), 
                    rochester['heat_source1'].sum()]

source2_thermal_y= [london['heat_source2'].sum(), 
                    stuttgart['heat_source2'].sum(), 
                    madrid['heat_source2'].sum(), 
                    rome['heat_source2'].sum(), 
                    miami['heat_source2'].sum(), 
                    fresno['heat_source2'].sum(), 
                    olympia['heat_source2'].sum(), 
                    rochester['heat_source2'].sum()]

source3_aquifer= [london['heat_aquifer'].sum(), 
                  stuttgart['heat_aquifer'].sum(), 
                  madrid['heat_aquifer'].sum(), 
                  rome['heat_aquifer'].sum(), 
                  miami['heat_aquifer'].sum(), 
                  fresno['heat_aquifer'].sum(), 
                  olympia['heat_aquifer'].sum(), 
                  rochester['heat_aquifer'].sum()]

source1_thermal_y =pd.Series(source1_thermal_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
source2_thermal_y =pd.Series(source2_thermal_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
source3_aquifer_y=pd.Series(source3_aquifer, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])

waste_heat_y=source1_thermal_y+source2_thermal_y


pd.DataFrame([thermal_y,
              electric_y,
              waste_heat_y,
              source3_aquifer_y,
              heat_loss_y,
              source1_thermal_y,
              source2_thermal_y,
             thermal_energy_network,],
             index=["thermal_demand", 
                    "electricity", 
                    "waste_heat", 
                    "ground_source", 
                    "thermal_loss",
                    "source1", 
                    "source2",
                   "thermal_energy_network"]).T["thermal_energy_network"]

In [ ]:
thermal_load_y=[london['Total_consumption'].sum(), stuttgart['Total_consumption'].sum(), madrid['Total_consumption'].sum(), rome['Total_consumption'].sum(), miami['Total_consumption'].sum(), fresno['Total_consumption'].sum(), olympia['Total_consumption'].sum(), rochester['Total_consumption'].sum()]
thermal_y =pd.Series(thermal_load_y)
thermal_y

electric_cons_y=[london['E_el'].sum(), stuttgart['E_el'].sum(), madrid['E_el'].sum(),rome['E_el'].sum(), miami['E_el'].sum(), fresno['E_el'].sum(), olympia['E_el'].sum(), rochester['E_el'].sum()]
electric_y =pd.Series(electric_cons_y)
electric_y

heat_loss_y=[london['E_loss_tot'].sum(), stuttgart['E_loss_tot'].sum(), madrid['E_loss_tot'].sum(),rome['E_loss_tot'].sum(), miami['E_loss_tot'].sum(), fresno['E_loss_tot'].sum(), olympia['E_loss_tot'].sum(), rochester['E_loss_tot'].sum()]
heat_loss_y =pd.Series(heat_loss_y)
heat_loss_y

thermal_energy_network= thermal_y+heat_loss_y+electric_y
thermal_energy_network =pd.Series(thermal_energy_network)

thermal_peak=[london['Total_consumption'].max()/1000, stuttgart['Total_consumption'].max()/1000, madrid['Total_consumption'].max()/1000, rome['Total_consumption'].max()/1000, miami['Total_consumption'].max()/1000, fresno['Total_consumption'].max()/1000, olympia['Total_consumption'].max()/1000, rochester['Total_consumption'].max()/1000]
thermal_peak_MW=pd.Series(thermal_peak)


COP=thermal_y/electric_y
COP

source1_thermal_y= [london['heat_source1'].sum(), stuttgart['heat_source1'].sum(), madrid['heat_source1'].sum(), rome['heat_source1'].sum(), miami['heat_source1'].sum(), fresno['heat_source1'].sum(), olympia['heat_source1'].sum(), rochester['heat_source1'].sum()]
source2_thermal_y= [london['heat_source2'].sum(), stuttgart['heat_source2'].sum(), madrid['heat_source2'].sum(), rome['heat_source2'].sum(), miami['heat_source2'].sum(), fresno['heat_source2'].sum(), olympia['heat_source2'].sum(), rochester['heat_source2'].sum()]
source3_aquifer= [london['heat_aquifer'].sum(), stuttgart['heat_aquifer'].sum(), madrid['heat_aquifer'].sum(), rome['heat_aquifer'].sum(), miami['heat_aquifer'].sum(), fresno['heat_aquifer'].sum(), olympia['heat_aquifer'].sum(), rochester['heat_aquifer'].sum()]


source1_thermal_y =pd.Series(source1_thermal_y)
source2_thermal_y =pd.Series(source2_thermal_y)
source3_aquifer_y=pd.Series(source3_aquifer)

waste_heat_y=source1_thermal_y+source2_thermal_y
COP


In [ ]:
london.columns

## Waste heat potential by city
There are many aspects of potential use of waste heat sources. A theoretical potential reflects the maximum output of the source if working all the hours in a year at its max. capacity. Instead, when considering the temporal allocation, its maximum output is constrained by the actual hours operation in which the source can be exploited.
Moreover, the availability of waste heat does not necessarily match the heat demand, and therefore its practical use is limited by this mismatch.
Visualization can be found here: https://public.flourish.studio/visualisation/3742380/
Rome case: https://public.flourish.studio/visualisation/3742465/

In [ ]:
london['source1_cap'].max()*london['hourofyear'].max()

In [ ]:
source1_theoretical=[london['source1_cap'].max()*london['hourofyear'].max(), stuttgart['source1_cap'].max()*stuttgart['hourofyear'].max(), madrid['source1_cap'].max()*madrid['hourofyear'].max(), rome['source1_cap'].max()*rome['hourofyear'].max(), miami['source1_cap'].max()*miami['hourofyear'].max(), fresno['source1_cap'].max()*fresno['hourofyear'].max(), olympia['source1_cap'].max()*olympia['hourofyear'].max(), rochester['source1_cap'].max()*rochester['hourofyear'].max()]
source2_theoretical=[london['source2_cap'].max()*london['hourofyear'].max(), stuttgart['source2_cap'].max()*stuttgart['hourofyear'].max(), madrid['source2_cap'].max()*madrid['hourofyear'].max(), rome['source2_cap'].max()*rome['hourofyear'].max(), miami['source2_cap'].max()*miami['hourofyear'].max(), fresno['source2_cap'].max()*fresno['hourofyear'].max(), olympia['source2_cap'].max()*olympia['hourofyear'].max(), rochester['source2_cap'].max()*rochester['hourofyear'].max()]

source1_theoretical_y =pd.Series(source1_theoretical)
source2_theoretical_y =pd.Series(source2_theoretical)

source1_technical=[london['source1_cap'].sum(), stuttgart['source1_cap'].sum(), madrid['source1_cap'].sum(), rome['source1_cap'].sum(), miami['source1_cap'].sum(), fresno['source1_cap'].sum(), olympia['source1_cap'].sum(), rochester['source1_cap'].sum()]
source2_technical=[london['source2_cap'].sum(), stuttgart['source2_cap'].sum(), madrid['source2_cap'].sum(), rome['source2_cap'].sum(), miami['source2_cap'].sum(), fresno['source2_cap'].sum(), olympia['source2_cap'].sum(), rochester['source2_cap'].sum()]

source1_technical_y =pd.Series(source1_technical)
source2_technical_y =pd.Series(source2_technical)

source1_practical_y= [london['heat_source1'].sum(), stuttgart['heat_source1'].sum(), madrid['heat_source1'].sum(), rome['heat_source1'].sum(), miami['heat_source1'].sum(), fresno['heat_source1'].sum(), olympia['heat_source1'].sum(), rochester['heat_source1'].sum()]
source2_practical_y= [london['heat_source2'].sum(), stuttgart['heat_source2'].sum(), madrid['heat_source2'].sum(), rome['heat_source2'].sum(), miami['heat_source2'].sum(), fresno['heat_source2'].sum(), olympia['heat_source2'].sum(), rochester['heat_source2'].sum()]


source1_practical_y =pd.Series(source1_practical_y)
source2_practical_y =pd.Series(source2_practical_y)


theoretical_potential= source1_theoretical_y+ source2_theoretical_y
technical_potential= source1_technical_y+ source2_technical_y
practical_potential=source1_practical_y + source2_practical_y

print(theoretical_potential, technical_potential, practical_potential)


In [ ]:
visualize_on_same_row(data_list=[miami[["space_heating_temp","hot_water_temp"]],
                                 fresno[["space_heating_temp","hot_water_temp"]],
                                 olympia[["space_heating_temp","hot_water_temp"]],
                                 rochester[["space_heating_temp","hot_water_temp"]],
                                 ospitaletto[["space_heating_temp","hot_water_temp"]]],
                      ylabel="Temperature in Celsius", 
                      xlabel="Date",
                      suptitle='Calculated Space Heating and Hot Water temperatures')

In [ ]:
month = int(input("Enter month to analyze (values between 1 to 12): "))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.month == month ]
november_data[["space_heating_temp","hot_water_temp"]].plot(figsize=(10,6), color=["#4e97a2",'#82272e'])

plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
Tamb_hourly["user_temp"].plot(figsize=(10,6))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.index.month == 11 ]
november_data["Tuser"].plot(figsize=(10,6))

## Visualization of Net temperature

In [ ]:
Tamb_hourly[["net_temp"]][Tamb_hourly.index < "2010-01-31"].plot()

## Visualization of COP

In [ ]:
Tamb_hourly[['COP']].plot(figsize=(20,6))

## Visualization of Energy Loss

In [ ]:
Tamb_hourly[['E_loss_tot']].plot(figsize=(20,6))

## Space Heating Distribution Plot

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()